# Introducción a los modelos de regresión — Parte 2
---


## 1. Preámbulo/encabezado

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

## 2. Datos

Datos de calibración de un FlexiForce.

In [ ]:
v_prom = np.array([0, 0.21, 0.48, 1.12, 1.27, 1.31, 1.43, 1.7, 1.92, 2.24, 2.3, 2.41, 2.53])  # [voltios]
masa   = np.array([10, 20, 50, 100, 120, 150, 200, 250, 300, 350, 400, 450, 500])             # [gramos]

## 3. Entrenamiento del modelo de regresión

### 3.1 Selección del orden del modelo
Modifique `orden` a 1, 2, 3, ... y ejecute nuevamente el notebook para observar el efecto.

In [ ]:
# Selección del orden del polinomio (equivalente a MATLAB "orden")
orden = 1

# scikit-learn requiere X en formato 2D: (n_muestras, n_características)
X = masa.reshape(-1, 1)
y = v_prom

# Pipeline: características polinómicas + regresión lineal
modelo = make_pipeline(
    PolynomialFeatures(degree=orden, include_bias=False),
    LinearRegression()
)

# Entrenamiento
modelo.fit(X, y)

### 3.2 Parametros del modelo

In [ ]:
# Parámetros del modelo: v(m) = b0 + b1*m + b2*m^2 + ... + b_orden*m^orden
lin = modelo.named_steps["linearregression"]

b0 = lin.intercept_
b = lin.coef_  # [b1, b2, ..., b_orden] porque include_bias=False

print(f"Orden: {orden}")
print(f"b0: {b0:.6f}")
for i, bi in enumerate(b, start=1):
    print(f"b{i}: {bi:.6e}")

### 3.4 Predicción del modelo

In [ ]:
# Respuesta del modelo
x = np.linspace(masa.min(), masa.max(), 20).reshape(-1, 1)
v_regresion = modelo.predict(x)

## 4. Visualización de resultados

Gráfica de los datos de calibración y la curva de regresión.

In [ ]:
plt.figure()
plt.plot(masa, v_prom, 'o', label='Datos de calibración')
plt.grid(True)

# Curva de regresión
plt.plot(x.ravel(), v_regresion, '-', label='Regresión')

plt.title('Datos de calibración y curva de regresión')
plt.xlabel('masa [gramos]')
plt.ylabel('voltaje [voltios]')
plt.legend()
plt.show()

## 5. Discusión

Ejecute nuevamente el notebook modificando el orden del polinomio (`orden = 1, 2, 3, ...`).

**Responda y justifique**:

1. ¿Cuáles son los parámetros (coeficientes y orden) del modelo que mejor ajustan a los datos?  
   Compare sus modelos usando al menos una métrica objetiva (por ejemplo, RMSE y/o \(R^2\)) y también inspección visual.

2. ¿Qué desafíos enfrenta al tratar de identificar el modelo óptimo?  
   Considere:
   - sensibilidad al ruido y variabilidad de las mediciones,
   - cantidad limitada de puntos de calibración,
   - inestabilidad numérica en polinomios de alto orden,
   - compromiso entre simplicidad y precisión.

3. Al incrementar el orden del polinomio, ¿qué fenómeno percibe? 

4. ¿Está familiarizado con el término *overfitting* o sobre-entrenamiento?
   Explique cómo puede aparecer en regresión polinómica y por qué puede perjudicar la predicción en datos nuevos.

### 5.1 Pregunta adicional
¿Cuál es el orden máximo del polinomio?
Con \(n\) puntos distintos, el grado máximo que puede interpolar todos los puntos suele ser \(n-1\).  
Sin embargo, en la práctica, grados muy altos pueden producir:
- problemas de condicionamiento (matrices casi singulares),
- oscilaciones fuertes entre puntos (fenómeno de Runge),
- mala generalización.

En este conjunto de datos, \(n = 13\), por lo tanto \(n-1 = 12\) sería el máximo “interpolante”, pero no suele recomendarse para calibración.